In [1]:
import numpy as np
import tqdm
import random
import scipy.misc
import os
import numpy.random as rng
from sklearn.utils import shuffle
import tensorflow as tf
import keras
import keras.backend as K
import scipy
from keras.applications.vgg16 import VGG16
from keras.models import Sequential,Model
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten, MaxPooling2D
from keras.regularizers import l2
from keras import optimizers

/root/miniconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
##Some part of the code has written with reference to https://github.com/ascourge21/Siamese
##Our proposed VGG16 augmented siamese network:
input_shape = (64,64,3)
left_input = Input(input_shape)
right_input = Input(input_shape)
VGG=VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(64,64,3), pooling=None)
model = Sequential()
for i in range(19):
    model.add(VGG.get_layer(index=i))
model.add(Flatten(input_shape=VGG.output_shape[1:]))
model.add(Dense(4096, activation='sigmoid',kernel_regularizer=l2(0.0001)))
first_twin = model(left_input)
second_twin = model(right_input)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[19:21]:
    layer.trainable = True
L1_layer =Lambda(lambda x: K.abs(x[0]-x[1]))
L1_distance = L1_layer([first_twin, second_twin])
prediction = Dense(1,activation='sigmoid')(L1_distance)
Siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
#optimizer=optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False )
optimizer = keras.optimizers.Adam(0.0001)
Siamese_net.compile(optimizer=optimizer, loss="binary_crossentropy")
Siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         23107392    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[1][0]               
          

In [39]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_24
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_8
20 dense_15


In [3]:
class0=os.path.join(os.getcwd() +'/clothes/train_rgb/0/' )
class1=os.path.join(os.getcwd() +'/clothes/train_rgb/1/' )
class2=os.path.join(os.getcwd() +'/clothes/train_rgb/2/' )
class3=os.path.join(os.getcwd() +'/clothes/train_rgb/3/' )
class4=os.path.join(os.getcwd() +'/clothes/train_rgb/4/' )
class5=os.path.join(os.getcwd() +'/clothes/train_rgb/5/' )
class6=os.path.join(os.getcwd() +'/clothes/train_rgb/6/' )
Class0=os.path.join(os.getcwd() +'/face/valid_rgb/0/' )
Class1=os.path.join(os.getcwd() +'/face/valid_rgb/1/' )
Class2=os.path.join(os.getcwd() +'/face/valid_rgb/2/' )
Class3=os.path.join(os.getcwd() +'/face/valid_rgb/3/' )
Class4=os.path.join(os.getcwd() +'/face/valid_rgb/4/' )
Class5=os.path.join(os.getcwd() +'/face/valid_rgb/5/' )
Class6=os.path.join(os.getcwd() +'/face/valid_rgb/6/' )
_0=os.path.join(os.getcwd() +'/face/test_split_rgb/0/' )
_1=os.path.join(os.getcwd() +'/face/test_split_rgb/1/' )
_2=os.path.join(os.getcwd() +'/face/test_split_rgb/2/' )
_3=os.path.join(os.getcwd() +'/face/test_split_rgb/3/' )
_4=os.path.join(os.getcwd() +'/face/test_split_rgb/4/' )
_5=os.path.join(os.getcwd() +'/face/test_split_rgb/5/' )
_6=os.path.join(os.getcwd() +'/face/test_split_rgb/6/' )
AN=os.path.join(os.getcwd() +'/Jpface/AN/' )
FE=os.path.join(os.getcwd() +'/Jpface/FE/' )
HA=os.path.join(os.getcwd() +'/Jpface/HA/' )
NE=os.path.join(os.getcwd() +'/Jpface/NE/' )
SA=os.path.join(os.getcwd() +'/Jpface/SA/' )
SU=os.path.join(os.getcwd() +'/Jpface/SU/' )
DI=os.path.join(os.getcwd() +'/Jpface/DI/' )
#50% same 50% different class ,randomly sampled pairs.
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = scipy.misc.imread(os.path.join(folder,filename))
        images.append(img)
    return images
## Different list represents two approaches for training: 3 classes of animated facial expression dataset and 
## 7 classes of Japanese female dataset
#Train=[load_images(AN),load_images(DI),load_images(FE),load_images(HA),load_images(NE),load_images(SA),load_images(SU)]
#train_image=[load_images(Class0),load_images(Class1),load_images(Class2)]
#test_image=[load_images(_0),load_images(_1),load_images(_2),load_images(_3),load_images(_4),load_images(_5),load_images(_6)]
#train_acc_image=[load_images(_0),load_images(_1),load_images(_2)]
#Test_image=[load_images(_3),load_images(_4),load_images(_5),load_images(_6)]

In [12]:

#-----------------------------------------------------------------------------------------------------------------
"""Create batch of 42 training pairs, 21 pairs with same class, 21 pairs with different class"""
batch_size=36
w, h = 64,64
pairs=[np.zeros((batch_size, w, h,3)) for i in range(2)]
#Remember to change setting when using face as train data:
catagory_11=np.array([0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2]) #catagory when using 3 class of face as train data
idx=np.random.choice(900,batch_size,replace=False)
#catagory_11=np.array([0,0,0,0,0,0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,5,5,5,5,5,5,5,6,6,6,6,6,6,6])
#catagory_11=np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2, \
#                      3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,\
#                      6,6,6,6,6,6,6,6,6,6,6,6,6,6])
"np.random.seed(10)"
"idx=np.random.choice(6000,42,replace=False)"
"Code commented are implemented separately to avoid fixed seed for random processes in future codes."

#idxx=[1789, 5566, 1657, 3983, 5619, 1933,  449,  492, 1172, 2040, 1638,
#2864,   43, 5540, 3758, 5606, 4936, 3399, 5436, 2602,  708, 4879,
#5593, 4736, 1324,  952, 3036, 4971, 2630, 2543, 4627, 1044,  502,
#1477, 5107, 4155, 2501, 3975,  143, 2083, 2514, 2569]
#b=list(np.arange(0,6000))
#for i in idxx:
#    b.remove(i)


"Since we are only using 42 pairs of photos, we can treat remaning entire training images as validation set. There is now"
"two separate dataset for training and validation."
#-------------------------------------------------------------------------------------------------------------------------
# Note that catagory,pairs,targets,idx in the definitions below changes based on the approach used.
def load_batch_train_pairs(batch_size):
    pairs=[np.zeros((batch_size, 64, 64,3)) for i in range(2)]
    targets=np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    a=np.random.choice(900,6,replace=False)
    idx=np.random.choice(900,36,replace=False)
    #idx=np.hstack((a,a,a,a,a,a,a,a,a,a,a,a,a,a))
    catagory_12=np.hstack((np.random.choice([1,2],size=(6,)), \
            np.random.choice([0,2],size=(6,)),\
            np.random.choice([0,1],size=(6,))))
    #catagory_12=np.hstack((np.random.choice([1,2,3,4,5,6],size=(14,)), \
    #        np.random.choice([0,2,3,4,5,6],size=(14,)), \
    #        np.random.choice([0,1,3,4,5,6],size=(14,)), \
    #        np.random.choice([0,1,2,4,5,6],size=(14,)), \
    #        np.random.choice([0,1,2,3,5,6],size=(14,)), \
    #        np.random.choice([0,1,2,3,4,6],size=(14,)), \
    #        np.random.choice([0,1,2,3,4,5],size=(14,))))
    for i in range(batch_size):
        if i<batch_size//2:
            pairs[0][i,:,:,:]=valid_image[catagory_11[i]][idx[i]]
            pairs[1][i,:,:,:]=valid_image[catagory_12[i]][idx[i]]
        if i>=batch_size//2:
            pairs[0][i,:,:,:]=valid_image[catagory_11[i-(batch_size//2)]][idx[i]] 
            pairs[1][i,:,:,:]=valid_image[catagory_11[i-(batch_size//2)]][idx[i]]
    return pairs,targets
def get_one_shot(n_ex,N,image_set):
    "n_ex = total no. of examples"
    "N = no.of class/N classes to distinguish from"
    categories = rng.choice(range(3),size=(N,),replace=False)            
    true_category = categories[0]
    index=rng.choice(n_ex,replace=False,size=(N,))
    ex1, ex2 = rng.choice(n_ex,replace=False,size=(2,))
    test_image_set = np.asarray([image_set[true_category][ex1]]*N)
    support_set=np.zeros((N,64,64,3))
    for i in range(1,N):
        support_set[i,:,:,:] = image_set[categories[i]][index[i]]
        support_set[0,:,:,:] = image_set[true_category][ex2]
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image_set, support_set = shuffle(targets, test_image_set, support_set) #Shuffle and give the same random orders of 
    #correct & incorrect pairs in three sets. 
    pairs = [test_image_set,support_set]
    return pairs,targets
def oneshot_accuracy(model,k,N,n_ex,image_set,verbose=0):
    "Calculation for accuracy of one-shot task to validation/test set after training the model"
    n_correct=0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ...".format(k,N))
    for i in range(k):
        inputs,targets=get_one_shot(n_ex,N,image_set)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
                n_correct+=1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% over {} random {} way one-shot learning accuracy".format(percent_correct,k,N))
    return percent_correct

In [14]:
val_acc = oneshot_accuracy(Siamese_net,320,7,6000,test_image,verbose=True)
print(val_acc)
##Major results:
##first approach:using first 3 class of facial expression as dataset.
##For 20 epoch training
#45%,47.185%,41.5625,44,375,40.625,49.0625,42.1875,49.375,41.6525,43.75 
##Second Approach: Japanese female facial expression (7 classes 213 photos in total)
##For 50epoch training on japanese face dataset(same image in the same class),100epoch,batch_size=98:
##28.4375%,25.625%accuracy
##Increasing training times seem do not improve accuracy furthermore .

Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 22.8125% over 320 random 7 way one-shot learning accuracy
22.8125


In [ ]:
n_iter=50
batch_size=36
eval_step=5
for z in range(1, n_iter):
    (inputs,targets)=load_batch_train_pairs(batch_size)
    loss=Siamese_net.train_on_batch(inputs,targets)
    print(loss)
    if n_iter%eval_step==0:
        train_acc = oneshot_accuracy(Siamese_net,320,3,6000,train_acc_image,verbose=True)
        print(train_acc)

In [ ]:
Results from previous cell:
1.26344
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 49.6875% over 320 random 3 way one-shot learning accuracy
49.6875
0.998732
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 56.5625% over 320 random 3 way one-shot learning accuracy
56.5625
0.920142
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 59.6875% over 320 random 3 way one-shot learning accuracy
59.6875
0.896159
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 62.8125% over 320 random 3 way one-shot learning accuracy
62.8125
0.886204
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 64.375% over 320 random 3 way one-shot learning accuracy
64.375
0.881523
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 64.375% over 320 random 3 way one-shot learning accuracy
64.375
0.881079
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 60.625% over 320 random 3 way one-shot learning accuracy
60.625
0.874748
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 65.625% over 320 random 3 way one-shot learning accuracy
65.625
0.870141
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 64.0625% over 320 random 3 way one-shot learning accuracy
64.0625
0.866885
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 68.4375% over 320 random 3 way one-shot learning accuracy
68.4375
0.863283
Evaluating model on 320 random 3 way one-shot learning tasks ...
Got an average of 63.4375% over 320 random 3 way one-shot learning accuracy
63.4375
0.860576
Evaluating model on 320 random 3 way one-shot learning tasks ...

In [12]:
val_acc = oneshot_accuracy(Siamese_net,320,7,6000,test_image,verbose=True)
print(val_acc)

n_iter=15
batch_size=196
eval_step=3
for z in range(1, n_iter):
    (inputs,targets)=load_batch_train_pairs(batch_size)
    loss=Siamese_net.train_on_batch(inputs,targets)
    print(loss)
    if n_iter%eval_step==0:
        val_acc = oneshot_accuracy(Siamese_net,320,7,6000,test_image,verbose=True)
        print(val_acc)
    #print("evaluating")
    #val_acc = oneshot_accuracy(siamese_net,100,7,900,valid_image,verbose=True)
    #print(val_acc)


Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 26.25% over 320 random 7 way one-shot learning accuracy
26.25
0.866949
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 27.1875% over 320 random 7 way one-shot learning accuracy
27.1875
0.861861
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 26.25% over 320 random 7 way one-shot learning accuracy
26.25
0.858603
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 21.25% over 320 random 7 way one-shot learning accuracy
21.25
0.852847
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 21.25% over 320 random 7 way one-shot learning accuracy
21.25
0.849759
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an average of 27.5% over 320 random 7 way one-shot learning accuracy
27.5
0.845707
Evaluating model on 320 random 7 way one-shot learning tasks ...
Got an av

KeyboardInterrupt: 

In [13]:
n_iter=25
batch_size=196
for z in range(1, n_iter):
    (inputs,targets)=load_batch_train_pairs(batch_size)
    loss=Siamese_net.train_on_batch(inputs,targets)
    print(loss)
    #print("evaluating")
    #val_acc = oneshot_accuracy(siamese_net,100,7,900,valid_image,verbose=True)
    #print(val_acc)



0.822696
0.81893
0.815235
0.811667
0.808363
0.804517
0.801224
0.797266
0.794281
0.790188
0.786906
0.7835
0.780472
0.776521
0.774148
0.769965
0.766446
0.763028
0.759882
0.756644
0.753691
0.750119
0.746706
0.743668
